# Search Magic Function Experimentation

## Desired Save Usage

```py
sand = SearchHandler()
sand.processor = processor
sand['description'] = "Description goes here"
sand['name'] = "Name goes here"
sand['category'] = "Category goes here"
sand['derp'] = ['list', 'of', 'tags']

sand.save() # this would automatically save a record
```

## Here's How Updates Would Function

```py
sand = SearchHandler()
sand.processor = processor
sand['description'] = "Description goes here"
sand['name'] = "Name goes here"
sand['category'] = "Category goes here"
sand['derp'] = ['list', 'of', 'tags']
sand.find() # This would be the query function. Query is too generic. We're gonna replace mongodb's formatting.
sand.replacement['description'] = "Description goes heresss"
sand.update() # this would automatically replace the top record we have from the find function.

# if we have a set of document ids we can iterate through them and update the status by finding
doc_ids = [uuid.uuid4().hex for _ in range(ids)]
sand = SearchHandler()
sand.processor = processor
sand.replacement['status'] = "FILLED"
for _id in doc_ids:
    sand.doc_id = _id
    sand.update() # it would find and update the exact field for the given id.
```

## Create Subcategories

For things like metadata we would create a massive subcategory field that we could query through.


```py
sand['subcategories'] = {
    "country": "US",
    "market": "crypto",
    "exchange": "coinbase",
    "loc": {
        "type": "GEO"
        "is_filter": False,
        "values": {
            "long": 33,
            "lat": -10,
            "distance": 1,
            "metric": "km"
        }
        
    }
}

# would turn into:

sub_index = f'{requirements_hash}:subcategories'
```

It would go through and find all of the field types. If it's a dictionary, we'd look through to find a `type` key. If there's not a `type` key, we'd skip it. We'd then add an extra value where we can relate the subcategory information directly to the main key.

In [15]:
from jamboree.utils.support.search.validation import is_nested, is_gen_type, name_match, is_generic, is_geo, to_str, to_field

In [16]:
from jamboree.utils.core import consistent_hash

In [26]:
class BaseSearchHandlerSupport(object):
    def __init__(self):
        self._requirements_str = {
            
        }
        self._subkey_names = set()
        self._indexable = set()
        self._index_key:str = ""
        self._sub_fields = {}
    
    @property
    def indexable(self):
        return list(self._indexable)
    
    @property
    def subnames(self):
        return self._subkey_names
    @property
    def index(self):
        """Index key for the requirements"""
        return self._index_key
    
    @property
    def subfields(self):
        return self._sub_fields
    
    def process_subfields(self):
        for key in self.subnames:
            self._sub_fields[key] = f"{self.index}:{key}"
    
    def process_requirements(self, _requirements:dict):
        """
            Process the required fields. That includes:
            
            1. Creating a requirements string. That's so we can create a key representing the field that exist.
            2. Listing all of the subkeys that we'd need to take in consideration.
            3. Creating an index hash to locate all relavent documents
            4. Creation of a list of fields so we can create a schema at that index hash
            5. Creation of all subkeys so we can quickly access them by name later
            
        """
        for k, v in _requirements.items():
            if is_generic(v):
                sval = to_str(v)
                self._requirements_str[k] = sval
                field = to_field(k, sval)
                self._indexable.add(to_field(k, sval))
                continue
                
            if v == dict:
                self._requirements_str[k] = "SUB"
                self.subnames.add(k)
                continue

            if is_geo(v):
                self._requirements_str[k] = "GEO"
                self._indexable.add(to_field(k, "GEO"))
                continue
        self._index_key = consistent_hash(self._requirements_str)
        self.process_subfields()

In [27]:
class BaseSearchHandler(BaseSearchHandlerSupport):
    def __init__(self):
        super().__init__()
        self._entity = None
        # This will only be here as an example
        self.requirements = {
            "name": str,
            "category": str,
            "subcategories": dict
        }
        
        # self.insert_builder = InsertBuilder()
        # self.query_builder = QueryBuilder()
        
        # Subs are all of the subfields we would need to search through
        self.subs = {}
        # Replacement is a set of fields we'd place in place of the ones we query or find by id
        self.replacement = {}
        
    
    @property
    def entity(self):
        if self._entity is None:
            raise AttributeError("You haven't set entity yet")
        return entity
    
    @entity.setter
    def entity(self, _entity:str):
        self._entity = _entity
    
    @property
    def requirements(self):
        return self._requirements_str
    
    @requirements.setter
    def requirements(self, _requirements:dict):
        """If we set it here we'd go through each dict item and create string version of each key"""
        _requirements['entity'] = str
        self.process_requirements(_requirements)
    
    def find(self, alt={}):
        """Given the items we've set, find all matching items"""
        pass
    
    def update(self, alt={}):
        """
            # Update
            Given the items or ID we've set, partial update every matching document. 
            If we have the document_ids already, replace those items
        """
#         insertion_dict = {}
#         self.processor.update(insertion_dict)
    
    def insert(self, alt={}):
        """
            # Insert
            Given all of the items we've set, add documents
        """
        insertion_dict = {}
        
        # self.processor.insert(insertion_dict)
        pass
    
    def remove(self, alt={}):
        pass
    
    def reset(self):
        """Reset all local variables"""
        pass
#     def __setitem__(self, key, value):
        
#         pass

In [28]:
base_handler = BaseSearchHandler()
base_handler.requirements

{'name': 'TEXT', 'category': 'TEXT', 'subcategories': 'SUB', 'entity': 'TEXT'}

In [29]:
base_handler.subfields

{'subcategories': 'eyJjYXRlZ29yeSI6IlRFWFQiLCJlbnRpdHkiOiJURVhUIiwibmFtZSI6IlRFWFQiLCJzdWJjYXRlZ29yaWVzIjoiU1VCIn0=:subcategories'}